In [1]:
## PARA O KC
# FAO https://www.fao.org/4/X0490E/x0490e0b.htm#crop%20coefficients

# PARA O CN 
# Capítulo 12 Infiltração usando o Método do número da curva CN do SCS
# https://engenhariacivilfsp.wordpress.com/wp-content/uploads/2014/05/infiltrac3a7c3a3o-usando-o-mc3a9todo-do-nc3bamero-da-curva.pdf

# Produção de base vetorial com o Curve Number (CN) para BHO 2014 (BHO_CN)
# https://metadados.snirh.gov.br/geonetwork/srv/api/records/d1c36d85-a9d5-4f6a-85f7-71c2dc801a67/attachments/NOTA_TECNICA_46_2018_SPR.pdf#page=7.17

In [6]:
# ALLEN, R. G. et al. Crop Evapotranpiration: Guidelines for computing crop water requirements. 
# Utah - USA: FAO - Food and Agriculture Organization of the United Nations, 1998. v. 56.  
# https://www.fao.org/4/X0490E/x0490e0b.htm

In [6]:
import pandas as pd
import geopandas as gpd
from osgeo import gdal, ogr
import numpy as np

In [9]:
def polygonize_raster(raster_path, shapefile_output):
    """
    Polygonize a raster file and save the output as a shapefile.

    Parameters:
    raster_path (str): Path to the input raster file (e.g., .tif).
    shapefile_output (str): Path to the output shapefile (e.g., .shp).

    Returns:
    GeoDataFrame: A GeoDataFrame containing the polygonized vector data.
    """
    
    raster = gdal.Open(raster_path)
    driver = ogr.GetDriverByName('GeoJSON')
    vector_ds = driver.CreateDataSource(shapefile_output)
    layer = vector_ds.CreateLayer("polygonized", geom_type=ogr.wkbPolygon)
    field = ogr.FieldDefn('lucode', ogr.OFTInteger)
    layer.CreateField(field)
    gdal.Polygonize(raster.GetRasterBand(1), None, layer, 0, [], callback=None)
    vector_ds = None
    gdf = gpd.read_file(shapefile_output)
    return gdf

In [10]:
years = list(range(1985, 2025))
res_esp = 30

In [11]:
all_dn = []
for year in years:
    print(year)
    lulc_dn = polygonize_raster(f"./lulc_data/land_use_{res_esp}m_{year}.tif", f"./lulc_data/land_use_{res_esp}m_{year}.geojson")
    all_dn.append(list(lulc_dn.lucode.unique()))

1985


/usr/lib/python3/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [12]:
list(set(np.concatenate(all_dn)))

[0, 33, 3, 4, 39, 41, 9, 11, 12, 46, 15, 48, 21, 24, 25]

In [13]:
#list(set(np.concatenate(all_dn)))
all_dn = [np.int32(0), np.int32(33), np.int32(3), np.int32(4), np.int32(39), np.int32(41), np.int32(9), np.int32(11), np.int32(12),
 np.int32(46), np.int32(15), np.int32(48), np.int32(21), np.int32(24), np.int32(25)]

In [14]:
def fill_biophysical_table(df, classes_id, descricao, kc_1, kc_2, kc_3, kc_4, kc_5, kc_6, kc_7, kc_8, kc_9, kc_10, kc_11, kc_12, cn_a, cn_b, cn_c, cn_d):
    for class_id in classes_id:
        valores_a_preencher = {'lucode': class_id, 'description': descricao,
            'Kc_1': kc_1,'Kc_2': kc_2, 'Kc_3': kc_3, 'Kc_4': kc_4, 'Kc_5': kc_5, 'Kc_6': kc_6, 
            'Kc_7': kc_7, 'Kc_8': kc_8, 'Kc_9': kc_9, 'Kc_10': kc_10, 'Kc_11': kc_11, 'Kc_12': kc_12,
            'CN_A': cn_a, 'CN_B': cn_b, 'CN_C': cn_c, 'CN_D': cn_d}
        df.loc[df['lucode'] == class_id, valores_a_preencher.keys()] = list(valores_a_preencher.values())
    return df

In [15]:
# https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2024/08/Legenda-Colecao-9-Descricao-Detalhada-PDF-PT.pdf
dn_to_name = pd.read_excel("codigos-da-legenda-colecao-9.xlsx")
dn_to_name = dn_to_name[dn_to_name.Class_ID.isin(set(all_dn))]

In [16]:
dn_to_name["Nivel "].unique()

array(['Floresta', 'Vegetação Herbácea e Arbustiva', 'Pastagem',
       'Agricultura', 'Silvicultura', 'Mosaico de Usos',
       'Área não Vegetada', "Corpo D'Água"], dtype=object)

In [17]:
dn_to_name[["Class_ID", "Nivel ", "Descricao"]][dn_to_name["Nivel "] == "Corpo D'Água"]

,Class_ID,Nivel,Descricao
34,33,Corpo D'Água,"Rio, Lago e Oceano"


In [18]:
## TABELA BIOFICA PARA TODOS OS USOS DO SOLO ENCONTRADOS ENTRE 1985 E 2023

In [19]:
# me preocupar em acertar kc e cn na seguinte ordem (% do LULC na bacia toda)
# ['Formação Savânica' 56,'Área Urbanizada' 26,'Formação Florestal' 8,'Pastagem' 6,
# 'Formação Campestre' 5,'Lago e Oceano' 5,'Mosaico de Usos' 5,'Outras Áreas não Vegetadas' 4,
#  'Campo Alagado e Área Pantanosa' < 1,'Soja' < 1, 'Outras Lavouras Temporárias' < 1, 'Silvicultura' < 1, 'Outras Lavouras Perenes' < 1, 'Café' < 1]

In [20]:
biophysical_table = dn_to_name[["Class_ID",  "Descricao"]].rename(columns = {"Class_ID": "lucode",  "Descricao": "description"}).drop_duplicates()

In [21]:
for i in range(1, 13): 
    biophysical_table[f'Kc_{i}'] = None 

for label in ['A', 'B', 'C', 'D']: 
    biophysical_table[f'CN_{label}'] = None

In [22]:
## Floresta
# Formação Florestal
biophysical_table = fill_biophysical_table(biophysical_table, [3], 'Formação Florestal',
                                           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ## ANJINHO
                                           36, 60, 73, 79) ## ANJINHO # SARTORI/ DE ABREU MARQUES - 20, 40, 49, 52
# Formação Savânica
biophysical_table = fill_biophysical_table(biophysical_table, [4], 'Formação Savânica', 
                                           0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, ## ANJINHO
                                           ## 35, 56, 70, 77 BENDITO
                                           39, 61, 74, 80) ## ANJINHO
## Vegetação Herbácea e Arbustiva
# Campo Alagado e Área Pantanosa
biophysical_table = fill_biophysical_table(biophysical_table, [11], "Campo Alagado e Área Pantanosa", 
                                           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # BENDITO
                                           39, 61, 74, 80) ## FALTAAAAAAAAAAAAAAAA
# Formação Campestre
biophysical_table = fill_biophysical_table(biophysical_table, [12], "Formação Campestre", 
                                          0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, ## ANJINHO
                                          39, 61, 74, 80) ## ANJINHO

## Pastagem
biophysical_table = fill_biophysical_table(biophysical_table, [15], 'Pastagem', 
                                           0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, ## ANJINHO
                                           49, 69, 79, 84) ## ANJINHO
## Agricultura
# Café, Outras Lavouras Perenes
biophysical_table = fill_biophysical_table(biophysical_table, [48], "Outras Lavouras Perenes", 
                                           0.69, 0.69, 0.69, 0.5, 0.5, 0.29, 0.29, 0.29, 0.5, 0.69, 0.69, 0.69, ## ANJINHO
                                           65, 75, 82, 86) ## ANJINHO
# Café, Outras Lavouras Perenes
biophysical_table = fill_biophysical_table(biophysical_table, [46], "Café", 
                                           0.69, 0.69, 0.69, 0.5, 0.5, 0.29, 0.29, 0.29, 0.5, 0.69, 0.69, 0.69, ## ANJINHO
                                           65, 75, 82, 86) ## ANJINHO
# Outras Lavouras Temporárias
biophysical_table = fill_biophysical_table(biophysical_table, [41], "Outras Lavouras Temporárias",
                                           0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8,  ## ANJINHO
                                           67, 78, 85, 89) ## ANJINHO
biophysical_table = fill_biophysical_table(biophysical_table, [39], "Soja", 
                                           0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8,  ## ANJINHO
                                           67, 78, 85, 89) ## ANJINHO
## Silvicultura
biophysical_table = fill_biophysical_table(biophysical_table, [9], "Silvicultura",
                                           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  ## ANJINHO
                                           36, 70, 73, 79)  ## ANJINHO
## Mosaico de Usos
biophysical_table = fill_biophysical_table(biophysical_table, [21], 'Mosaico de Usos', 
                                           0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, # ANJINHO
                                           67, 78, 85, 89) ## ANJINHO
## Área não Vegetada
## Área Urbanizada
biophysical_table = fill_biophysical_table(biophysical_table, [24], "Área Urbanizada", 
                                           0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ## ANJINHO
                                           98, 98, 98, 98) ## ANJINHO
## Outras Áreas não Vegetadas
biophysical_table = fill_biophysical_table(biophysical_table, [25], "Outras Áreas não Vegetadas", 
                                           0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ## ANJINHO
                                           77, 86, 91, 94) ## ANJINHO
## Corpo D'Água
biophysical_table = fill_biophysical_table(biophysical_table, [33], "Rio, Lago e Oceano", 
                                           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ## ANJINHO
                                          99, 99, 99, 99) # ANJINHO

In [23]:
biophysical_table.to_csv("./biophysical_data/biophysical_table_30m_all_years.csv", index = False)

In [24]:
biophysical_table.to_excel("./biophysical_data/biophysical_table_30m_all_years.xlsx", index = False)

In [25]:
## para cada ano

In [26]:
biophysical_table = pd.read_excel("./biophysical_data/biophysical_table_30m_all_years.xlsx")

In [27]:
res_esp = "30"
years = [str(y) for y in range(1985, 2025)]
for year in years:
    lulc_dn = gpd.read_file(f"./lulc_data/land_use_{res_esp}m_{year}.geojson")
    lulc_dn = lulc_dn[lulc_dn.lucode !=0].reset_index(drop=True)
    biophysical_table[biophysical_table.lucode.isin(list(lulc_dn["lucode"].unique()))].to_csv(f"./biophysical_data/biophysical_table_{res_esp}m_{year}.csv", index = False)